In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data=pd.read_csv('/kaggle/input/crime-data/01_District_wise_crimes_committed_IPC_Till_2013.csv')
data.head()

In [4]:
data.shape

In [5]:
data.info()

In [ ]:
data.describe()

In [4]:
totaldata=pd.read_csv('../input/crime-data/01_District_wise_crimes_committed_Total_IPC_Till_2013.csv')
totaldata.head()

All india's Total crime rate in each year

In [24]:
statepopulation=pd.read_csv('../input/crime-data/State_population.csv')
statepopulation['Population (2011 Census)[11]']=statepopulation['Population (2011 Census)[11]'].str.replace(',','')
statepopulation.head()

In [7]:
totaldata.info()

In [6]:
mergedata=pd.read_csv('../input/crime-data/01_District_wise_crimes_committed_Merge_IPC_Till_2013.csv')
mergedata.head()

In [7]:
crime_types=mergedata.columns[3:-1]
crime_types

In [23]:
a='12,13'
a.replace(',','')

In [26]:
denominator=int(statepopulation[statepopulation['STATE/UT']=='ASSAM']['Population (2011 Census)[11]'])
newdata=mergedata.loc[mergedata['STATE/UT']=='ASSAM']
newdata['MURDER']=newdata['MURDER'].apply(lambda i : i/denominator)
newdata

In [28]:
suggestions={'MURDER':'Keep objects of self defence with you in public. Be prepared to defend yourself at all times. Remember emergency numbers. Do not engage the attackers unless it is absolutely unavoidable.  ',
            'RAPE':'Travel in well-lit, well-traveled areas. If possible, walk in pairs. Walk facing traffic. Plan your route ahead of time. Know your neighborhood - be aware of nearby businesses, their hours of operation and their locations. Avoid shortcuts, bushy areas, and alleyways. Dress for ease of movement.',
            'KIDNAPPING':'Don’t discuss your family with strangers. Do proper checks before employing workers. Don’t get too close to a stranger. Keep emergency numbers ready',
            'DACOITY':'During late night hours, request a stop by asking the driver to let you off anywhere along the route, even if it is not a designated stop. Use well-lit and busy stops whenever possible. If you’re getting off at a little-used stop, try to arrange for a friend to meet with you. When riding the bus or waiting at a bus stop stay alert, and don’t doze off.',
            'ROBBERY':' Carry your bag in front of you. Gentlemen, carry your wallet in your front pocket.',
             'FRAUD':'Pay the safest way. Guard your personal information. Don’t believe promises of easy money. Check money notes carefully',
            }
suggestions

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [52]:
state='ANDHRA PRADESH'
crime='RAPE'

moving_data=list(mergedata.loc[mergedata['STATE/UT']==state][crime])
curdata=mergedata.loc[mergedata['STATE/UT']==state][[crime,'YEAR']]
for year in range(2014,2023):
    value=sum(moving_data[-5:])/5
    nowyear=[value,year]
    curdata=pd.concat([curdata,pd.DataFrame({crime:[value],'YEAR':[year]})],axis=0)
    moving_data.append(value)

curdata.plot(x='YEAR',y=crime,kind='line')
plt.show()

In [31]:
from_state='ASSAM'
to_state='ANDHRA PRADESH'
N=5

for crime_type in crime_types:
    from_data=mergedata.loc[mergedata['STATE/UT']==from_state]
    to_data=mergedata.loc[mergedata['STATE/UT']==to_state]
    fd=int(statepopulation[statepopulation['STATE/UT']==from_state]['Population (2011 Census)[11]'])
    td=int(statepopulation[statepopulation['STATE/UT']==to_state]['Population (2011 Census)[11]'])
    from_data[crime_type]=from_data[crime_type].apply(lambda i: i/fd)
    to_data[crime_type]=to_data[crime_type].apply(lambda i: i/td)
    
    ax=from_data[crime_type].plot(kind='line',y='TOTAL IPC CRIMES')
    to_data[crime_type].plot(kind='line',y='TOTAL IPC CRIMES',ax=ax,color='red')
    plt.title('Graph based on '+crime_type)
    plt.show()
    
    next_from_year=from_data.tail(N)[crime_type].sum()/N
    next_to_year=to_data.tail(N)[crime_type].sum()/N
    print('ASSAM next year possible '+crime_type+' value is :',next_from_year)
    print('ANDHRA next year possible '+crime_type+' value is :',next_to_year)
    if(next_to_year>next_from_year):
        print(crime_type+' On andra is more than your from district')
        if(crime_type in suggestions):
            print()
            print(suggestions[crime_type])
            print()
            print()

In [ ]:
totaldata.columns

In [ ]:
df_by_year = totaldata.groupby(by='YEAR').sum()

df_by_year.plot(kind='bar',y='TOTAL IPC CRIMES')
df_by_year.plot(kind='line',y='TOTAL IPC CRIMES')


#plt.title("Bar chart of Total no of IPC crimes vs year",fontdict={'size':15,'fontfamily':'Times New Roman'})
#plt.ylabel("Total No of IPC Crimes",fontdict={'size':13,'fontfamily':'DejaVu Sans'})
#plt.xlabel("Year",fontdict={'size':14,'fontfamily':'DejaVu Sans'})
plt.show()

In [ ]:
df_by_state = totaldata[totaldata['is_UT']==0].groupby(by ='STATE/UT').sum()

fig,ax=plt.subplots(figsize=(20,10))
plt.tick_params(labelsize=14)
plt.xticks(label = df_by_state.index,rotation=90)
ax.bar(x=df_by_state.index,height=df_by_state['TOTAL IPC CRIMES'])
ax.set_title("Bar graph of total no of IPC crimes Vs States",fontdict={'size':26,'fontfamily':"Britannic Bold"})
ax.set_ylabel("Total no of IPC crimes",fontdict={'size':20,'fontfamily':"DejaVu Sans"})
ax.set_xlabel("States",fontdict={'size':20,'fontfamily':"DejaVu Sans"})

plt.show()

In [ ]:
df_by_ut = totaldata[totaldata['is_UT']==1].groupby(by ='STATE/UT').sum()

fig,ax=plt.subplots(figsize=(5,5))
plt.tick_params(labelsize=14)
plt.xticks(label = df_by_ut.index,rotation=90)
ax.bar(x=df_by_ut.index,height=df_by_ut['TOTAL IPC CRIMES'])
ax.set_title("Bar graph of total no of IPC crimes Vs UT",fontdict={'size':26,'fontfamily':"Britannic Bold"})
ax.set_ylabel("Total no of IPC crimes",fontdict={'size':20,'fontfamily':"DejaVu Sans"})
ax.set_xlabel("Union Territories",fontdict={'size':20,'fontfamily':"DejaVu Sans"})

plt.show()

In [ ]:
df_by_city = data.groupby(by='DISTRICT').sum()
#df_by_city = df_by_city[df_by_city.index != 'TOTAL']
df_by_city.drop('YEAR',axis =1,inplace=True)
df_by_city.sort_values(by='TOTAL IPC CRIMES',ascending = False,inplace=True)
total= df_by_city['TOTAL IPC CRIMES'].sum()

df_by_city['IPC_pct'] = df_by_city['TOTAL IPC CRIMES'].apply(lambda x: x*100/total)

In [ ]:
df_Delhi = totaldata[totaldata['STATE/UT'] == 'DELHI UT']

fig,ax=plt.subplots(figsize=(15,10))
plt.tick_params(labelsize=14)
plt.xticks(label = df_Delhi['YEAR'],rotation=90)
ax.bar(x=df_Delhi['YEAR'],height=df_Delhi['RIOTS'])
ax.set_facecolor('1')
ax.set_title("Bar graph of total no Riots in Delhi Vs Year",fontdict={'size':26,'fontfamily':"Britannic Bold"})
ax.set_ylabel("Total no of Riots",fontdict={'size':20,'fontfamily':"DejaVu Sans"})
ax.set_xlabel("Year",fontdict={'size':20,'fontfamily':"DejaVu Sans"})

plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(15,10))
plt.tick_params(labelsize=14)
plt.xticks(label = df_Delhi['YEAR'],rotation=90)
ax.barh(y=df_Delhi['YEAR'],width =df_Delhi['RAPE'])
ax.set_facecolor('1')
ax.set_title("Bar graph of total no Rapes in Delhi Vs Year",fontdict={'size':26,'fontfamily':"Britannic Bold"})
ax.set_ylabel("Total no of Rapes",fontdict={'size':20,'fontfamily':"DejaVu Sans"})
ax.set_xlabel("Year",fontdict={'size':20,'fontfamily':"DejaVu Sans"})

plt.show()

In [ ]:
df_by_state = totaldata[totaldata['is_UT']==0].groupby(by ='STATE/UT').sum()

fig,ax=plt.subplots(figsize=(20,10))
plt.tick_params(labelsize=14)
plt.xticks(label = df_by_state.index,rotation=90)
ax.bar(x=df_by_state.index,height=df_by_state['TOTAL IPC CRIMES'])
ax.set_title("Bar graph of total no of IPC crimes Vs States",fontdict={'size':26,'fontfamily':"Britannic Bold"})
ax.set_ylabel("Total no of IPC crimes",fontdict={'size':20,'fontfamily':"DejaVu Sans"})
ax.set_xlabel("States",fontdict={'size':20,'fontfamily':"DejaVu Sans"})

plt.show()

YEAR BAR : 2001-2013 or ALL or ACROSS

CRIME BAR : Murder-Other_IPC_Crimes or ALL or ACROSS

Start with whole india, divide into states and then to districts 

In [ ]:
#FOR WHOLE INDIA
year_type=input('Enter Year Type')
crime_type=input('Enter Crime Type')

if(year_type=='ACROSS'):
    df_by_year = totaldata.groupby(by='YEAR').sum()
    if(crime_type=='ALL'):
        df_by_year.plot(kind='bar',y='TOTAL IPC CRIMES')
        df_by_year.plot(kind='line',y='TOTAL IPC CRIMES')
    elif(crime_type!='ACROSS'):
        df_by_year[crime_type].plot(kind='bar',y='TOTAL IPC CRIMES')
        df_by_year[crime_type].plot(kind='line',y='TOTAL IPC CRIMES')
    else:
        df_by_year=df_by_year.loc[:,df_by_year.columns!='is_UT']
        df_by_year.plot(kind='line')
elif(crime_type=='ACROSS'):
    if(year_type=='ALL'):
        totaldata[all_crimes].sum().plot(kind='bar')
        totaldata[all_crimes].sum().plot(kind='line')
    elif(year_type!='ACROSS'):
        totaldata.loc[totaldata['YEAR']==int(year_type)][all_crimes].sum().plot(kind='bar')
        totaldata.loc[totaldata['YEAR']==int(year_type)][all_crimes].sum().plot(kind='line')
    else:
        pass
else:
    pass

In [ ]:
#ACROSS STATE WISE
df_by_year = totaldata.loc[totaldata['STATE/UT']=='ANDHRA PRADESH']
df_by_year.plot(x='YEAR',y='TOTAL IPC CRIMES', kind='bar')

In [ ]:
df1 = totaldata.loc[totaldata['STATE/UT']=='ANDHRA PRADESH'].copy()
df1['NEW PREDICTED']=df1['TOTAL IPC CRIMES'].rolling(window=12).mean()
df1

In [ ]:
totaldata.groupby(by='STATE/UT').sum().head()

In [ ]:
totaldata.to_csv('01_District_wise_crimes_committed_Total_IPC_Till_2013.csv',index=False)
data.to_csv('01_District_wise_crimes_committed_IPC_Till_2013.csv',index=False)

In [ ]:
#Attempt to murder same as attempt to commit murder
#Burglary same as avg('Criminal Trespass/Burglary', 'Criminal Trespass or Burglary')
#counterfieting same as Counterfeiting
#Hurt/grevious hurt same as sum('Grievous Hurt', 'Hurt')
#importation of girls from foreign countries same as 'Importation of Girls from Foreign Country'
#insult to modesty of women same as Insult to the Modesty of Women
#kidnapping and abduction of others same as Other Kidnapping
#kidnapping and abduction of women and girls same as Kidnapping & Abduction of Women to compel her for marriage
#other rape same as Rape_Others
#other theft same as Other Thefts
#preparation ane assembly for dacoity same as Making Preparation and Assembly for committing Dacoity
#state/UT same as 'States/UTs'
#total ipc crimes same as Total Cognizable IPC crimes